#### Задача: Финансовый отчет

Задание: Создать отчет с колонками:date:
- Тип Date. Битая дата должна быть удалена (строка удаляется)
- revenue: Тип Double. Если Null -> 0
- tax: Тип Double. Если Null -> 0
- net_profit: revenue - tax.

Критерий: В итоговом DataFrame не должно быть Null значений в цифрах и не должно быть строк с пустой датой._str"]

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("Data_Cleaning").getOrCreate()

dirty_financials = [
    ("2023-01-01", "1000", "10"),
    ("2023-01-01", None, "10"),  # Нет суммы
    ("N/A", "500", "5"),         # Битая дата
    ("2023-01-02", "200.50", None) # Нет налога
]
cols = ["date_str", "revenue_str", "tax_str"]

df = spark.createDataFrame(dirty_financials, schema=cols)
df.show()
df.printSchema()

+----------+-----------+-------+
|  date_str|revenue_str|tax_str|
+----------+-----------+-------+
|2023-01-01|       1000|     10|
|2023-01-01|       NULL|     10|
|       N/A|        500|      5|
|2023-01-02|     200.50|   NULL|
+----------+-----------+-------+

root
 |-- date_str: string (nullable = true)
 |-- revenue_str: string (nullable = true)
 |-- tax_str: string (nullable = true)



In [10]:
# Тип Date. Битая дата должна быть удалена (строка удаляется)
df_date = df.withColumn(
    "date_str",
    F.to_date(F.col("date_str"), "yyyy-MM-dd")
).na.drop(subset=["date_str"])

df_date = df_date.withColumnRenamed("date_str", "date")

df_date.show()

+----------+-----------+-------+
|      date|revenue_str|tax_str|
+----------+-----------+-------+
|2023-01-01|       1000|     10|
|2023-01-01|       NULL|     10|
|2023-01-02|     200.50|   NULL|
+----------+-----------+-------+



In [18]:
df_final = df_date.withColumns({
    "revenue": F.col("revenue_str").cast("double"),
    "tax": F.col("tax_str").cast("double")
}).na.fill(value=0, subset=["revenue", "tax"]).withColumn(
    "net_profit", 
    F.col("revenue") - F.col("tax")
).select(F.col("date"), F.col("revenue"), F.col("tax"), F.col("net_profit"))

df_final.show()


+----------+-------+----+----------+
|      date|revenue| tax|net_profit|
+----------+-------+----+----------+
|2023-01-01| 1000.0|10.0|     990.0|
|2023-01-01|    0.0|10.0|     -10.0|
|2023-01-02|  200.5| 0.0|     200.5|
+----------+-------+----+----------+

